In [1]:
import pandas as pd
import openai
from openai.types.beta.threads.message_create_params import (
    Attachment,
    AttachmentToolFileSearch,
)

import picologging as logging

import pathlib
import datetime
import asyncio
from asyncio import Semaphore
import json


In [2]:
with open("../tokens/openai_token.txt") as f:
    openai_token = f.read()

In [3]:
now = datetime.datetime.now()
now_str = now.strftime("%Y%m%d_%H%M%S")
pathlib.Path(f"../notebooks_logging/06_ask_q4pages").mkdir(parents=True, exist_ok=True)
# pathlib.Path(f"../notebooks_logging/05_cut_pdfs/{now_str}").mkdir(parents=True, exist_ok=True)

file_handler = logging.FileHandler(f"../notebooks_logging/06_ask_q4pages/{now_str}.log")
stream_handler = logging.StreamHandler()
# stdout_formatter = logging.Formatter(fmt="%(message)s") 
# stream_handler.setFormatter(stdout_formatter)
logging.basicConfig(
    encoding='utf-8', 
    format='%(asctime)s | %(levelname)s | %(message)s', 
    level=logging.INFO, 
    datefmt='%Y-%m-%dT%H:%M:%S',
    handlers=[
        file_handler,
        stream_handler,
    ],
    force=True
)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
li = logger.info
li("start")

2025-02-26T16:13:48 | INFO | start


In [6]:
df_temp_answers = pd.read_parquet(f"../data_temp/04 temp_answers.parquet")
df_temp_answers = df_temp_answers.rename(columns={"documents_to_look_through": "ref_sha1"})
df_temp_answers

,q_text,q_kind,company_names,company_names_subset,ref_sha1
0,"According to the annual report, what is the Op...",number,[Altech Chemicals Ltd],[Altech Chemicals Ltd],[63688d5d0b4f12e9f847c5407439a1ec46047a4a]
1,"According to the annual report, what is the Op...",number,[Cofinimmo],[Cofinimmo],[9cc771c2171bacc138cda4e7d68b8b427a514d81]
2,Did Cofinimmo outline any new ESG initiatives ...,boolean,[Cofinimmo],[Cofinimmo],[9cc771c2171bacc138cda4e7d68b8b427a514d81]
3,What is the total number of employees let go b...,number,"[Hagerty, Inc.]","[Hagerty, Inc.]",[a6f23184a87f3343f17e8e8ed08f604615cdefc1]
4,Which leadership **positions** changed at Reno...,names,[Renold plc],[Renold plc],[da8afefdc3840175c9f26a4dbbed05e250a342cc]
5,What was the Gross margin (%) for Charles & Co...,number,"[Charles & Colvard, Ltd.]","[Charles & Colvard, Ltd.]",[d3a834539b046a49708161a6c0d35aad29dd15ec]
6,What was the Capital expenditures (in GBP) for...,number,[Harworth Group plc],[Harworth Group plc],[ed55750eae9cb7d893e6484b92496639172717cd]
7,What was the Capital expenditures (in USD) for...,number,"[Charles & Colvard, Ltd.]","[Charles & Colvard, Ltd.]",[d3a834539b046a49708161a6c0d35aad29dd15ec]
8,What are the names of new products launched by...,names,[Zymeworks Inc.],[Zymeworks Inc.],[69b472e4c05986db72ff8e547bb220e2d222b7ef]
9,"For Lipocine Inc., what was the value of Numbe...",number,[Lipocine Inc.],[Lipocine Inc.],[c51f3c5aff7bea6fbc0bb7537838fa2f44f35c23]


In [5]:
df_subset = pd.read_parquet(f"../data_temp/04 subset.parquet")
df_subset

,sha1,cur,company_name,major_industry,mentions_recent_mergers_and_acquisitions,has_leadership_changes,has_layoffs,has_executive_compensation,has_rnd_investment_numbers,has_new_product_launches,...,has_dividend_policy_changes,has_share_buyback_plans,has_capital_structure_changes,mentions_new_risk_factors,has_guidance_updates,has_regulatory_or_litigation_issues,has_strategic_restructuring,has_supply_chain_disruptions,has_esg_initiatives,is_in_questions
0,0a9e39e1d2e176f3a766a0e86af82772f1654a6e,USD,KeyCorp,Financial Services,True,True,False,True,True,True,...,True,True,False,True,True,True,False,False,True,False
1,20c4badd4303f7aba1a298d84be3722fa84e0c67,USD,"Summit Materials, Inc.",Financial Services,True,True,False,True,False,False,...,False,True,False,True,False,False,False,True,True,False
2,30b729c124a24ff21f37431dab6b58dfe7ba56fa,USD,Gibson Energy Inc.,Energy and Utilities,False,False,False,True,False,False,...,True,True,False,False,False,True,False,False,False,False
3,42f03832077a92b3b34855cb7f9ef93563143838,EUR,HCW Biologics Inc.,Healthcare,True,False,False,True,True,True,...,False,False,False,True,True,True,False,False,False,False
4,49a9bf0542f3e5ff0250064bfed4369ecf6c8a09,USD,Essential Metals Limited,Technology,True,False,False,True,True,False,...,False,False,False,True,False,False,False,False,True,False
5,54e625a049a1713eb5a338b64858c06d74e52489,USD,"EVI Industries, Inc.",Retail,True,False,False,True,False,False,...,False,False,False,True,False,False,False,True,False,False
6,56262704bde6b584dcebfa644faa23a953498a79,EUR,Koenig & Bauer AG,Technology,True,True,False,True,True,True,...,False,False,False,True,True,False,True,True,True,False
7,58b196e02c2d9749d968a29039e9c2b29d3d31e1,USD,Microchip Technology Incorporated,Technology,True,True,False,True,True,True,...,False,True,True,True,True,True,False,True,True,False
8,5ac3ccdec033f81fab4c2ed9ae86553f4904a450,EUR,"Corbus Pharmaceuticals Holdings, Inc.",Pharmaceuticals,True,False,False,True,True,True,...,False,False,False,True,True,True,False,False,False,False
9,601aba58deffc81230c837404aa883de0d1dde1c,USD,Audalia Resources Limited,Technology,False,True,False,True,True,True,...,False,False,False,True,False,False,False,False,False,True


In [7]:
question_id, row = list(df_temp_answers.iterrows())[0]
row

question_text                According to the annual report, what is the Op...
kind                                                                    number
documents_to_look_through                                                 [13]
ref_sha1                            [63688d5d0b4f12e9f847c5407439a1ec46047a4a]
Name: 0, dtype: object

In [8]:
ref_sha1_i = row["ref_sha1"][0]

pdf_pages = list(pathlib.Path(f"../data_temp/05_cut_pdfs/{ref_sha1_i}/").glob('*'))
pdf_pages


[WindowsPath('../data_temp/05_cut_pdfs/63688d5d0b4f12e9f847c5407439a1ec46047a4a/63688d5d0b4f12e9f847c5407439a1ec46047a4a_0000.pdf'),
 WindowsPath('../data_temp/05_cut_pdfs/63688d5d0b4f12e9f847c5407439a1ec46047a4a/63688d5d0b4f12e9f847c5407439a1ec46047a4a_0001.pdf'),
 WindowsPath('../data_temp/05_cut_pdfs/63688d5d0b4f12e9f847c5407439a1ec46047a4a/63688d5d0b4f12e9f847c5407439a1ec46047a4a_0002.pdf'),
 WindowsPath('../data_temp/05_cut_pdfs/63688d5d0b4f12e9f847c5407439a1ec46047a4a/63688d5d0b4f12e9f847c5407439a1ec46047a4a_0003.pdf'),
 WindowsPath('../data_temp/05_cut_pdfs/63688d5d0b4f12e9f847c5407439a1ec46047a4a/63688d5d0b4f12e9f847c5407439a1ec46047a4a_0004.pdf'),
 WindowsPath('../data_temp/05_cut_pdfs/63688d5d0b4f12e9f847c5407439a1ec46047a4a/63688d5d0b4f12e9f847c5407439a1ec46047a4a_0005.pdf'),
 WindowsPath('../data_temp/05_cut_pdfs/63688d5d0b4f12e9f847c5407439a1ec46047a4a/63688d5d0b4f12e9f847c5407439a1ec46047a4a_0006.pdf'),
 WindowsPath('../data_temp/05_cut_pdfs/63688d5d0b4f12e9f847c5407439a1

In [9]:
pdf_page = str(pdf_pages[0])
pdf_page

'..\\data_temp\\05_cut_pdfs\\63688d5d0b4f12e9f847c5407439a1ec46047a4a\\63688d5d0b4f12e9f847c5407439a1ec46047a4a_0000.pdf'

In [10]:
client = openai.AsyncOpenAI(
    # This is the default and can be omitted
    api_key=openai_token,
)

In [11]:
q_text = row["question_text"]
q_text

"According to the annual report, what is the Operating margin (%) for Altech Chemicals Ltd  (within the last period or at the end of the last period)? If data is not available, return 'N/A'."

In [12]:
q_kind = row["kind"]
q_kind

'number'

In [13]:
prompt = f"""Please answer the following question: 
{q_text}

Check that the data is really present in the file.
Check that the found data cannot be confused with other things.

---
Output format:
The output should start with @@@ and end with @@@. If the result is empty it should still correspond to the JSON schema and the result should have N/A in result.
The output is a JSON that corresponds to the following schema:
""" + """
{
    "type": "object",
    "properties": {
        "result": {
            "description": "The answer to the question",
            "type": \"""" + q_kind + """\",
        },
        "result_prob": {
            "description": "Probability that the result is the right one. Float from 0 to 1.",
            "type": "number"
        },
        "result_found": {
            "description": "Is the answer to the question present in the pdf?",
            "type": "bool",
        },
        "result_found_prob": {
            "description": "Probability of result_found",
            "type": "number"
        },        
    },
    "required": [
        "result",
        "result_prob",
        "result_found",
        "result_found_prob",
    ]
}
"""


In [14]:
li(f"pdf_sha1={ref_sha1_i}")
li(f"q_text={q_text}")
li(f"q_kind={q_kind}")
li(f"prompt=\n{prompt}")

2025-02-25T20:25:12 | INFO | pdf_sha1=63688d5d0b4f12e9f847c5407439a1ec46047a4a
2025-02-25T20:25:12 | INFO | q_text=According to the annual report, what is the Operating margin (%) for Altech Chemicals Ltd  (within the last period or at the end of the last period)? If data is not available, return 'N/A'.
2025-02-25T20:25:12 | INFO | q_kind=number
2025-02-25T20:25:12 | INFO | prompt=
Please answer the following question: 
According to the annual report, what is the Operating margin (%) for Altech Chemicals Ltd  (within the last period or at the end of the last period)? If data is not available, return 'N/A'.

Check that the data is really present in the file.
Check that the found data cannot be confused with other things.

---
Output format:
The output should start with @@@ and end with @@@. If the result is empty it should still correspond to the JSON schema and the result should have N/A in result.
The output is a JSON that corresponds to the following schema:

{
    "type": "object",


In [20]:
sem = Semaphore(100)
async def send_one_page(page_i: int, pdf_page: str, prompt: str) -> openai.types.beta.threads.Run:
    async with sem:
        pdf_assistant = await client.beta.assistants.create(
            model="gpt-4o",
            # model="o1-mini",
            description="An assistant to extract the contents of PDF files.",
            tools=[{"type": "file_search"}],
            name="PDF assistant",
        )

        # li(f"Assistent created")

        # Create thread
        thread = await client.beta.threads.create()

        # li(f"thread created")

        file = await client.files.create(file=open(pdf_page, "rb"), purpose="assistants")
        # li(f"file created")

        # Create assistant
        await client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            attachments=[
                Attachment(
                    file_id=file.id, tools=[AttachmentToolFileSearch(type="file_search")]
                )
            ],
            content=prompt,
        )
        # li(f"message created")

        # Run thread
        run = await client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=pdf_assistant.id, timeout=1000
        )

        messages_cursor = await client.beta.threads.messages.list(thread_id=run.thread_id)
        messages = [message async for message in messages_cursor]

        # Output text
        res_txt = messages[0].content[0].text.value
        return run, page_i, res_txt

In [21]:

pages_done = ["0"]*len(pdf_pages)
pages_tries = {}
tasks = []
li(f"Sending for every page started")
for page_i, pdf_page in enumerate(pdf_pages):
    pages_tries[page_i] = 1
    tasks.append(asyncio.create_task(send_one_page(page_i, pdf_page, prompt)))

2025-02-25T20:29:26 | INFO | Sending for every page started


In [22]:
page_i_runs = []
for task in asyncio.as_completed(tasks):
    # get task result
    run, page_i, res_txt = await task
    # report result
    li(f"page_i={page_i}")
    if run.status == "completed":
        pages_done[page_i] = "1"
    page_i_runs.append((page_i, run, res_txt))
    li("".join(pages_done))

2025-02-25T20:29:37 | INFO | page_i=62
2025-02-25T20:29:37 | INFO | 00000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000
2025-02-25T20:29:37 | INFO | page_i=45
2025-02-25T20:29:37 | INFO | 00000000000000000000000000000000000000000000010000000000000000100000000000000000000000000000
2025-02-25T20:29:38 | INFO | page_i=61
2025-02-25T20:29:38 | INFO | 00000000000000000000000000000000000000000000010000000000000001100000000000000000000000000000
2025-02-25T20:29:38 | INFO | page_i=35
2025-02-25T20:29:38 | INFO | 00000000000000000000000000000000000100000000010000000000000001100000000000000000000000000000
2025-02-25T20:29:39 | INFO | page_i=73
2025-02-25T20:29:39 | INFO | 00000000000000000000000000000000000100000000010000000000000001100000000001000000000000000000
2025-02-25T20:29:39 | INFO | page_i=28
2025-02-25T20:29:39 | INFO | 00000000000000000000000000001000000100000000010000000000000001100000000001000000000000000000
2025-02-25T20:29:39 | INFO | page_

In [ ]:
# sem = Semaphore(20) -> 3m45s
# sem = Semaphore(100) -> 1m39s

In [23]:
page_i_runs_sorted = sorted(page_i_runs, key=lambda tup: tup[0])
page_i_runs_sorted[:2]

[(0,
  Run(id='run_VZ6r6b32A6LD1mzgJheLOmEB', assistant_id='asst_Z6lLEJFLip0QEySwx2k73xN2', cancelled_at=None, completed_at=1740504581, created_at=1740504574, expires_at=None, failed_at=None, incomplete_details=None, instructions=None, last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1740504575, status='completed', thread_id='thread_i5Q3JSGFCBNMUrgckE11FojG', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=100, prompt_tokens=2649, total_tokens=2749, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), temperature=1.0, top_p=1.0, tool_resource

In [25]:

statuses = [run.status for _, run, _ in page_i_runs_sorted]
check_res = (pd.Series(statuses) == "completed")
for i_checked, checked in enumerate(check_res.to_list()):
    if checked:
        pages_done[i_checked] = 1
check_res_01 = check_res * 1
check_res_01_string = "".join(check_res_01.astype(str).to_list())
check_res_01_string

'11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111'

In [26]:
res_txts = []

for _, _, res_txt in page_i_runs_sorted:
    res_txts.append(res_txt)


In [27]:
res_txts

['@@@\n{\n    "result": "N/A",\n    "result_prob": 0.0,\n    "result_found": false,\n    "result_found_prob": 1.0\n}\n@@@',
 '@@@\n{\n    "result": "N/A",\n    "result_prob": 0,\n    "result_found": false,\n    "result_found_prob": 0.9\n}\n@@@',
 '@@@\n{\n    "result": "N/A",\n    "result_prob": 0.0,\n    "result_found": false,\n    "result_found_prob": 0.9\n}\n@@@',
 '@@@\n{\n    "result": "N/A",\n    "result_prob": 0.0,\n    "result_found": false,\n    "result_found_prob": 0.0\n}\n@@@',
 '@@@\n{\n    "result": 0,\n    "result_prob": 0,\n    "result_found": false,\n    "result_found_prob": 0.9\n}\n@@@',
 '@@@\n{\n    "result": "N/A",\n    "result_prob": 0,\n    "result_found": false,\n    "result_found_prob": 0.9\n}\n@@@',
 '@@@\n{\n    "result": "N/A", \n    "result_prob": 0.0, \n    "result_found": false, \n    "result_found_prob": 1.0 \n}\n@@@',
 '@@@\n{\n    "result": "N/A",\n    "result_prob": 0,\n    "result_found": false,\n    "result_found_prob": 1\n}\n@@@',
 '@@@\n{\n    "res

In [28]:
res_txt2 = [x.replace("\n", "") for x in res_txts]
res_txt2

['@@@{    "result": "N/A",    "result_prob": 0.0,    "result_found": false,    "result_found_prob": 1.0}@@@',
 '@@@{    "result": "N/A",    "result_prob": 0,    "result_found": false,    "result_found_prob": 0.9}@@@',
 '@@@{    "result": "N/A",    "result_prob": 0.0,    "result_found": false,    "result_found_prob": 0.9}@@@',
 '@@@{    "result": "N/A",    "result_prob": 0.0,    "result_found": false,    "result_found_prob": 0.0}@@@',
 '@@@{    "result": 0,    "result_prob": 0,    "result_found": false,    "result_found_prob": 0.9}@@@',
 '@@@{    "result": "N/A",    "result_prob": 0,    "result_found": false,    "result_found_prob": 0.9}@@@',
 '@@@{    "result": "N/A",     "result_prob": 0.0,     "result_found": false,     "result_found_prob": 1.0 }@@@',
 '@@@{    "result": "N/A",    "result_prob": 0,    "result_found": false,    "result_found_prob": 1}@@@',
 '@@@{    "result": "N/A",    "result_prob": 0,    "result_found": false,    "result_found_prob": 0.9}@@@',
 'In the provided docu

In [29]:
out_string = "\n".join(res_txt2)
pathlib.Path(f"../notebooks_logging/06_ask_q4pages/{now_str}").mkdir(parents=True, exist_ok=True)
with open(f"../notebooks_logging/06_ask_q4pages/{now_str}/06_result.txt", "w") as f:
    f.write(out_string)

pathlib.Path(f"../data_temp/06_ask_q4pages/{question_id:04d}").mkdir(parents=True, exist_ok=True)
with open(f"../data_temp/06_ask_q4pages/{question_id:04d}/06_result.txt", 'w') as f:
    f.write(out_string)

li(f"../notebooks_logging/06_ask_q4pages/{now_str}/06_result.txt")
li(f"../data_temp/06_ask_q4pages/{question_id:04d}/06_result.txt")
li(f"number of results(lines)={len(res_txt2)}")

2025-02-25T20:34:48 | INFO | ../notebooks_logging/06_ask_q4pages/20250225_202512/06_result.txt
2025-02-25T20:34:48 | INFO | ../data_temp/06_ask_q4pages/0000/06_result.txt
2025-02-25T20:34:48 | INFO | number of results(lines)=92


In [31]:
res_txt3 = []
for line in res_txt2:
    try:
        ans = json.loads(line.split("@@@")[1])
    except:
        ans = {
            "result": None,
            "result_format_problem": True,
        }
    res_txt3.append(ans)
res_txt3[:5]

[{'result': 'N/A',
  'result_prob': 0.0,
  'result_found': False,
  'result_found_prob': 1.0},
 {'result': 'N/A',
  'result_prob': 0,
  'result_found': False,
  'result_found_prob': 0.9},
 {'result': 'N/A',
  'result_prob': 0.0,
  'result_found': False,
  'result_found_prob': 0.9},
 {'result': 'N/A',
  'result_prob': 0.0,
  'result_found': False,
  'result_found_prob': 0.0},
 {'result': 0,
  'result_prob': 0,
  'result_found': False,
  'result_found_prob': 0.9}]

In [32]:
df_q = pd.DataFrame(res_txt3)
df_q

,result,result_prob,result_found,result_found_prob
0,N/A,0.0,False,1.0
1,N/A,0.0,False,0.9
2,N/A,0.0,False,0.9
3,N/A,0.0,False,0.0
4,0,0.0,False,0.9
...,...,...,...,...
87,N/A,0.0,False,1.0
88,N/A,0.0,False,0.5
89,N/A,0.0,False,1.0
90,N/A,0.0,False,0.0


In [34]:
df_q[df_q["result_found"]==True]

,result,result_prob,result_found,result_found_prob


In [36]:
df_q["result"].value_counts()

result
N/A    87
0       2
Name: count, dtype: int64

In [37]:
df_q["result_prob"].value_counts()

result_prob
0.0    91
1.0     1
Name: count, dtype: int64

In [38]:
df_q[df_q["result_prob"]== 1.0]

,result,result_prob,result_found,result_found_prob
39,N/A,1.0,False,0.9
